*Group 30 Contributers: Zhong Tan, Nicholas Low, Stephen Huang* 

*To group 30 members: add your name here as proof of contribution*

To edit, go inside Jupyter. Click the diamond icon on the left bar. Click the cloud down arrow. Edit changes. Save. The project is now under [Changed]. Hover over it and click on the [+]. Fill in the summary with what you changed. Click [COMMIT]. Click the cloud up arrow. Everything is now updated.

**Using R and knn regression to predict used car costs in the US based on mileage, brand and year**

**Introduction**

Being able to predict used car prices can save families thousands of dollars. By using the progam language R, we can plot graphs and get a prediction for how much a car may cost. We can use a method called regression and predict a numerical value for a used car's price based on factors we enter.

We will predict the cost of a used car based on mileage and year. As background knowledge, we know that newer products with more technology cost more compared to older products with less features. We apply this knowledge to used cars in our prediction. We also know that used products are valued less than products in better conditions. Knowing that used or worn products are valued less is why we choose mileage as a factor. We believe that overall used car prices will be changed when we change mileage and year.

In our example, we use linear regression, as we want interpretability in our final results. In contrast, another option, KNN regression, tends to be more confusing to interpret. With linear regression, we might be able to predict the cost of a used car based on mileage and year. We will be using used, American car prices scraped from the web. We have taken this dataset and put it into our group's repository for easy access. To extract this dataset, we will start with read_csv() of our comma seperated sheet in .csv format, USA_cars_datasets.csv.

*Choose which RMSPE is lower in KNN vs linear regression*

**Preliminary exploratory analysis**

First we load our libraries and set our seed to make sure our work is reproducible.

In [1]:
install.packages("tidyverse")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [2]:
library(tidyverse)
library(digest)
library(repr)
library(tidymodels)
library(cowplot)
library(tidymodels)
library(GGally)
library(ISLR)
set.seed(0)
options(repr.matrix.max.rows = 8)

Okay, let's see our data.

In [ ]:
car <- read_csv("https://raw.githubusercontent.com/zhtgeneral/Group-30-Official/main/Data/USA_cars_datasets.csv")
head(car)

Our columns are number, price, brand, model, year, title, mileage, color, vin, lot, state, country and condition. We have 1 variable per column. We notice that we have 1 observation per row. This means we have a tidy dataset.

Since we're only looking at mileage and year, we will select those columns.

In [ ]:
car <- car |>
       select(price, year, mileage)
head(car)

This is our reduced data. With only 3 columns, it will be easier to work on it. Let's get an idea for the relationship between these variables. 

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 7)
car_year <- car |>
            ggplot(aes(x = price, y = year)) +
            geom_point() +
            ylim(1990, 2021) +
            labs(x = "Price",
                 y = "Year",
                 title = "Price vs Year of used cars in the US")
car_mileage <- car |>
            ggplot(aes(x = price, y = mileage)) +
            geom_point() +
            ylim(0, 400000) +
            labs(x = "Price",
                 y = "Mileage",
                 title = "Price vs Mileage of used cars in the US")
plot_grid(car_year, car_mileage, ncol = 2)

So these are the relationships between price and each of the variables. It seems plausible to use linear regression to represent the trends. Let's try both linear regression and KNN regression to see which RMSPE is lower and which model is more accurate.

In [ ]:
# Let's use linear regression while training it with our training data.
car_split <- initial_split(car, prop = 0.75, strata =price)
car_train <- training(car_split)
car_test <- testing(car_split)

car_recipe <- recipe(price ~ mileage, year, data = car_train) |>
              step_scale(all_predictors()) |>
              step_center(all_predictors())

lin_spec <- linear_reg() |>
            set_engine("lm") |>
            set_mode("regression")

lin_fit <- workflow() |>
           add_recipe(car_recipe) |>
           add_model(lin_spec) |>
           fit(data = car_train)

lin_car_results <- lin_fit |>
                   predict(car_test) |>
                   bind_cols(car_test) |>
                   metrics(truth = price, estimate = .pred) |>
                   filter(.metric == "rmse") |>
                   pull()

lin_car_results

This is our RMSPE when we use linear regression. Let's compare it to KNN regression.

In [ ]:
car_recipe <- recipe(price ~ mileage, year, data = car_train) |>
              step_scale(all_predictors()) |>
              step_center(all_predictors())

knn_spec <- nearest_neighbor(weight_func = "rectangular",
                             neighbors = tune()) |>
            set_engine("kknn") |>
            set_mode("regression")

knn_vfold <- vfold_cv(car_train, v = 5, strata = price)

knn_fit <- workflow() |>
           add_recipe(car_recipe) |>
           add_model(knn_spec)

grid_vals <- tibble(neighbors = seq(from = 1, to = 200, by = 5))

knn_car_results <- knn_fit |>
                    tune_grid(resamples = knn_vfold, grid = grid_vals) |>
                    collect_metrics() |>
                    filter(.metric == "rmse") |>
                    filter(mean == min(mean))  #Finding best K value for KNN regression
knn_car_results

We see that we want k = 96 in our linear regression. Let's try this and see the RMSPE from KNN regression and compare it to linear regression.

In [ ]:
knn_spec <- nearest_neighbor(weight_func = "rectangular",
                             neighbors = 96) |>
            set_engine("kknn") |>
            set_mode("regression")

knn_fit <- workflow() |>
           add_recipe(car_recipe) |>
           add_model(knn_spec) |>
           fit(data = car_train)

knn_car_summary <- knn_fit |>
                    predict(car_test) |>
                    bind_cols(car_test) |>
                    metrics(truth = price, estimate = .pred) |>
                    filter(.metric == "rmse")
knn_car_summary

So we have the RMSPE from KNN regression. This is lower than the  11327 from linear regression. This lower RMSPE value means we should use KNN regression.

**Methods**

To conduct our data analysis, we will look at only mileage, production year of the car, and the brand. The lot number, vin number, state, and the day the listing was uploaded doesn't make a difference. We can visualize our results with many scatterplots. We can use a scatterplot with price on the y axis and year/mileage on the x axis then have a bar plot with price on the x axis and brand on the y axis (like the graph above). We can also get a  measure of accuracy if we use a model called knn regression. As car brands have an effect on the price of cars (as a car from a reputable or luxury car brand will generally be marketed at a higher price than a car with the same specs from a different car brand), we will be separating the data by the car brand and performing regression on every brand individually. This way, we are able to predict car prices accounting for individual brands. We can then visualize these in a line graph and place then together using the patchwork package.

*KNN regression vs linear regression*

*Use GGPairs*

To begin, we must read and clean the data:

In [ ]:
#car_original_data <- read_csv("https://raw.githubusercontent.com/zhtgeneral/Group-30-Official/main/Data/USA_cars_datasets.csv")
#car_data_clean <- car_original_data |>
    #...
#mutate, filter, select, and skip as you see fit
#car_data_tidy

As can be seen from the tidy data set...

Now we can plot the data using the ggplot function, and add the linear regression model on top...

In [ ]:
#insert linear regression here...




#for the plots:
#car_predictions_brand1 <- car_best_fit_brand1 |>
 #   predict(car_data_training_brand1) |>
 #   bind_cols(car_data_training_brand1)
#car_predictions_brand1

#car_plot_brand1 <- car_predictions_brand1 |>
 #   ggplot(aes(x = specs, y = price)) +
 #   geom_point() +
 #   geom_line(mapping = aes(x = specs, y = .pred), color = "blue") +
 #   labs(x = "Specs", y = "Price (CAD)", color = "Predicted Time") +
 #   ggtitle("Figure 1: graph depicting the linear regression model that predicts the price of brand1 cars based off of their specs")
#car_plot_brand1

We can import the patchwork package to add the plots together:

In [ ]:
#library(patchwork)
#plot_1 + plot_2 + plot_3

**Expected Outcomes and significance**

We can expect used BMWs and Fords to cost more than the average used car. We can expect the cost of a random used car to be around between 6000 and $22000 USD based on the top 10 most abundant used cars. Being able to estimate the cost of a used car based on brand, mileage, and year can give us knowledge to know whether to wait for a better deal or to buy now. This analysis can bring insight to the question of how much can designer brand cars with high mileage be cheaper than economy brand cars with low mileage.

*Make sure to relate to intro*
